**Primeiramente, utilizei seis algoritmos para o treinamento:**

- LogisticRegression(), 
- RandomForestClassifier(), 
- GradientBoostingClassifier(), 
- SVC(), 
- CatBoostClassifier(),
- keras.Sequential()

**Após avaliar o desempenho de cada modelo com hiperparâmetros padrão, escolhi:**
 
- Redes Neurais
- Gradiente Boost
- Regressao Logisticas

Esses modelos seram selecionados para a tunagem de parametros.


**Para otimizar os hiperparâmetros dos modelos Gradiente Boost e Regressao Logisticas, utilizei o algoritmo RandomizedSearchCV.Essa abordagem usa Otimização Bayesiana passo a passo para explorar os hiperparâmetros mais promissores no espaço do problema. Para a Redes Neurais Sequenciais, utilizei o algoritmo keras_tuner.RandomSearch,ele vem com algoritmos de Otimização Bayesiana, Hiperbanda e Pesquisa Aleatória integrados e também foi projetado para ser fácil para pesquisadores estenderem a fim de experimentar novos algoritmos de pesquisa.**

# Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
import keras
import keras_tuner
from keras import layers
from sklearn.model_selection import RandomizedSearchCV
import warnings
import pickle
import warnings
import joblib
warnings.filterwarnings("ignore")

<frozen importlib._bootstrap>:488: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
2024-07-23 16:47:39.486706: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-23 16:47:39.563588: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-23 16:47:42.734116: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-23 16:47:52.722293: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA,

# Importacao dos dados

In [2]:
dados_tratados = pickle.load(open('dados/dados_tratados.pickle', 'rb'))

In [3]:
x_treino = dados_tratados['x_treino']
y_treino = dados_tratados['y_treino']
x_teste = dados_tratados['x_teste']
y_teste = dados_tratados['y_teste']

print('Shape dos dados de x_treino:', x_treino.shape)
print('Shape dos dados de y_treino:', y_treino.shape)
print('Shape dos dados de x_teste:', x_teste.shape)
print('Shape dos dados de y_teste:', y_teste.shape)

Shape dos dados de x_treino: (117218, 20)
Shape dos dados de y_treino: (117218,)
Shape dos dados de x_teste: (15891, 20)
Shape dos dados de y_teste: (15891,)


# Treinamento do modelo

## Selecionando os modelos 

In [4]:
## criar a rede seqeuncial

model_rede = keras.Sequential()
model_rede.add(layers.Dense(units=40,activation='relu',input_shape=(20,)))
#model.add(layers.Dropout(0.20))
model_rede.add(layers.Dense(units=60,activation='relu'))
#model.add(layers.Dropout(0.20))
model_rede.add(layers.Dense(units=40,activation='relu'))
#model.add(layers.Dropout(0.20))
model_rede.add(layers.Dense(units=1,activation='sigmoid'))
model_rede.summary()
opt = keras.optimizers.SGD(learning_rate=0.01)
model_rede.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy', 'Recall'])
#epochs_hist = model.fit(x_treino, y_treino.values, epochs=100,batch_size=500)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 40)             │           840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 60)             │         2,460 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 40)             │         2,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            41 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,781 (22.58 KB)

 Trainable params: 5,781 (22.58 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
##
modelos = {
    'logistic_regression': LogisticRegression(),
    'random_forest': RandomForestClassifier(),
    'gradient_boost': GradientBoostingClassifier(),
    'svc': SVC(),
    'cat_boost': CatBoostClassifier(),
    'redes_sequencial': model_rede
}

from sklearn.metrics import classification_report,recall_score
lista_recall = {}
dic_model = {}
for key,model in modelos.items():
    if key == 'redes_sequencial': 
        epochs_hist = model.fit(x_treino, y_treino, epochs=100,batch_size=500,verbose=0)  
        print('O modelo treinado é o:', key)
        y_pred = model.predict(x_teste).squeeze()
        y_pred_redes = (y_pred > 0.5).astype(int)
        lista_recall[key] = recall_score(y_teste,y_pred_redes)
        print(classification_report(y_teste, y_pred_redes))  
        dic_model[key] = model
        print('----------------------------------------------------')
        
    else:
        model = model
        if key == 'cat_boost':
            model_treinado = model.fit(x_treino, y_treino,verbose=0)
        else:
            model_treinado = model.fit(x_treino, y_treino)
        print('O modelo treinado e de:', key)
        print(key, model_treinado.score(x_teste, y_teste))
        print(classification_report(y_teste, model.predict(x_teste)))
        lista_recall[key] = recall_score(y_teste,model.predict(x_teste))
        dic_model[key] = model
        print('----------------------------------------------------')


O modelo treinado e de: logistic_regression
logistic_regression 0.967402932477503
              precision    recall  f1-score   support

           0       1.00      0.97      0.98     15518
           1       0.41      0.92      0.57       373

    accuracy                           0.97     15891
   macro avg       0.71      0.95      0.78     15891
weighted avg       0.98      0.97      0.97     15891

----------------------------------------------------
O modelo treinado e de: random_forest
random_forest 0.9886099049776603
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     15518
           1       0.86      0.62      0.72       373

    accuracy                           0.99     15891
   macro avg       0.92      0.81      0.86     15891
weighted avg       0.99      0.99      0.99     15891

----------------------------------------------------
O modelo treinado e de: gradient_boost
gradient_boost 0.9619281354225663
              

**Análise dos Resultados**

Os modelos que tem os três melhores desempenhos são:

- Redes Neurais
- Gradiente Boost
- Regressao Logisticas

Esses modelos seram selecionados para a tunagem de parametros.


In [21]:
pd.DataFrame(list(lista_recall.items()), columns=['Model', 'Recall']).sort_values(by='Recall', ascending=False)

,Model,Recall
2,gradient_boost,0.946381
0,logistic_regression,0.922252
5,redes_sequencial,0.879357
3,svc,0.863271
4,cat_boost,0.806971
1,random_forest,0.616622


# Tunagem dos modelos

## Gradiente boost

In [4]:
## Parametros Gradiente boost
parametros_grad = {"n_estimators": [2, 10, 20, 30, 50, 100, 200, 300, 400, 500],
                    "learning_rate": [0.1, 0.15, 0.2, 0.4, 0.6, 0.8, 1],
                    'max_features': [1, 3, 4,5,6,8,10],
                }

grid_grad = RandomizedSearchCV(
            GradientBoostingClassifier(verbose=0), parametros_grad, verbose=0, scoring="recall"
        )
grid_grad.fit(x_treino, y_treino)

RandomizedSearchCV(estimator=GradientBoostingClassifier(),
                   param_distributions={'learning_rate': [0.1, 0.15, 0.2, 0.4,
                                                          0.6, 0.8, 1],
                                        'max_features': [1, 3, 4, 5, 6, 8, 10],
                                        'n_estimators': [2, 10, 20, 30, 50, 100,
                                                         200, 300, 400, 500]},
                   scoring='recall')

In [5]:
print('Melhores parametros:',grid_grad.best_params_,'Score:',grid_grad.best_score_)

Melhores parametros: {'n_estimators': 500, 'max_features': 6, 'learning_rate': 0.2} Score: 1.0


In [6]:
grad_model = GradientBoostingClassifier(**grid_grad.best_params_)
grad_model.fit(x_treino, y_treino)

GradientBoostingClassifier(learning_rate=0.2, max_features=6, n_estimators=500)

In [7]:
joblib.dump(grad_model, 'modelos_salvos/gradiente_boost_model.pkl')

['modelos_salvos/gradiente_boost_model.pkl']

## Regrassao logistica

In [10]:
parametros_logistica = {'penalty': ['l2', 'none'],
            'tol': [0.0001,0.001,0.01,0.1],
            'C': [1.0, 2.0, 3.0, 4.0],
            
                 }
grid_logistica = RandomizedSearchCV(
            LogisticRegression(), parametros_logistica,verbose=0, scoring="recall"
        )
grid_logistica.fit(x_treino, y_treino)

RandomizedSearchCV(estimator=LogisticRegression(),
                   param_distributions={'C': [1.0, 2.0, 3.0, 4.0],
                                        'penalty': ['l2', 'none'],
                                        'tol': [0.0001, 0.001, 0.01, 0.1]},
                   scoring='recall')

In [11]:
logistica_model = LogisticRegression(**grid_logistica.best_params_)
logistica_model.fit(x_treino, y_treino)

LogisticRegression(C=3.0, tol=0.01)

In [12]:
joblib.dump(logistica_model, 'modelos_salvos/logistica_model.pkl')

['modelos_salvos/logistica_model.pkl']

## Redes Sequencial

In [20]:
def build_model(hp,camadas_ocultas=4):
    model = keras.Sequential()
    # Primeira camada com input_shape especificado corretamente
    model.add(keras.layers.Dense(
          hp.Choice('units', [8, 16, 32,50,64,100]),
          activation='relu',input_shape=(x_treino.shape[1],)))
    # Camadas ocultas adicionais
    for _ in range(camadas_ocultas):
        model.add(keras.layers.Dense(
            hp.Choice('units', [8, 16, 32,50, 64,100]),
            activation='relu'))
    # Camada de saída
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    # Definição do otimizador
    opt = keras.optimizers.SGD(learning_rate= hp.Choice("learning_rate", [0.001, 0.01, 0.02 ,0.05,0.1, 0.2, 0.3]))
    # Compilação do modelo
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', 'Recall'])
    return model

tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,)

In [ ]:
tuner.search(x_treino, y_treino, epochs=10,verbose=0)
best_model = tuner.get_best_models()[0]

In [28]:
best_model.save("modelos_salvos/redes_neurais_model.h5")

In [22]:
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,049 (70.50 KB)

 Trainable params: 18,049 (70.50 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
best_model.evaluate(x_treino, y_treino)

3664/3664 ━━━━━━━━━━━━━━━━━━━━ 45s 12ms/step - Recall: 0.6524 - accuracy: 0.9253 - loss: 0.3101


[0.8365392088890076, 0.8162056803703308, 0.6547629237174988]

In [26]:
best_model.evaluate(x_teste, y_teste)

497/497 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - Recall: 0.6951 - accuracy: 0.9714 - loss: 0.0844


[0.08902431279420853, 0.9696054458618164, 0.6782841682434082]